In [63]:
#Loading BP dataset
import pandas as pd
df = pd.read_excel('/Users/shrikantkale/Desktop/meditation_dataset.xlsx')

In [64]:
df.head(10)

,id,q1,q2,q3,q4,q5,q6,label_cat
0,1,1,1,1,1,1,1,focus_med
1,2,1,2,1,1,1,1,visual
2,3,1,1,2,1,1,1,inner_focus
3,4,1,1,1,2,1,1,focus_med
4,5,1,1,1,1,2,1,visual
5,6,1,2,2,1,1,1,visual
6,7,1,1,2,2,1,1,visual
7,8,1,1,1,2,2,1,focus_med
8,9,1,2,3,1,1,3,outer_focus
9,10,1,1,2,3,1,2,visual


In [65]:
df['label_cat'].value_counts()

focus_med      8
visual         7
outer_focus    7
mindful        6
inner_focus    3
Name: label_cat, dtype: int64

In [66]:
features = df.drop(['label_cat'], axis=1)
features.head(2)

,id,q1,q2,q3,q4,q5,q6
0,1,1,1,1,1,1,1
1,2,1,2,1,1,1,1


In [67]:
label = df[['label_cat']]
label.head(2)

,label_cat
0,focus_med
1,visual


In [68]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X, y = ros.fit_resample(features, label)

In [69]:
y['label_cat'].value_counts()

focus_med      8
visual         8
inner_focus    8
outer_focus    8
mindful        8
Name: label_cat, dtype: int64

In [70]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
encoder = OrdinalEncoder()
# transform data
y = encoder.fit_transform(np.array(y['label_cat']).reshape(-1,1))

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [72]:
from xgboost import XGBClassifier
model = XGBClassifier(objective="multi:softmax", tree_method='hist', max_depth=1500)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [73]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, predictions)
print(accuracy_score(y_test, predictions))

0.75


In [74]:
cm

array([[1, 0, 0, 0, 0],
       [0, 4, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [75]:
results = pd.DataFrame(y_test)
results['pred'] = predictions
results.head(10)

,0,pred
0,2.0,3
1,3.0,4
2,1.0,1
3,4.0,4
4,1.0,1
5,1.0,1
6,1.0,1
7,0.0,0


In [76]:
y = encoder.inverse_transform(np.array(results['pred']).reshape(-1,1))
y

array([['outer_focus'],
       ['visual'],
       ['inner_focus'],
       ['visual'],
       ['inner_focus'],
       ['inner_focus'],
       ['inner_focus'],
       ['focus_med']], dtype=object)

In [77]:
#pickle the model
import pickle
file_name ="xgb.pkl"
pickle.dump(model, open(file_name, "wb"))